# This code cleans the Rideshare Trip data 

It removes duplicates, trips under 2 miles, and trips over 1 day. 
It then exports the remaining data to a csv. Appropriate for 1 month of data at a time. 

In [1]:
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
import re

In [2]:
rs = pd.read_csv('data/Chicago/rideshare/Rideshare_M12.csv') #reads rideshare trip information as rs

In [3]:
rs

,Trip ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,...,Additional Charges,Trip Total,Shared Trip Authorized,Trips Pooled,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,2bb643fba9dbf5266f699c0c632bbeae50f0e655,12/01/2022 12:00:00 AM,12/01/2022 12:30:00 AM,1659,16.5,1.703198e+10,1.703181e+10,76.0,NaN,30.0,...,6.80,36.80,False,1,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),NaN,NaN,NaN
1,4f730568c6b9a1379decfdb38d81de05e38eb140,12/01/2022 12:00:00 AM,12/01/2022 12:45:00 AM,2825,33.2,1.703106e+10,NaN,6.0,NaN,62.5,...,2.10,64.60,False,1,41.942577,-87.647079,POINT (-87.6470785093 41.942577185),NaN,NaN,NaN
2,0066be3a1237becddbdcdaa54304ff8971e528b7,12/01/2022 12:00:00 AM,12/01/2022 12:15:00 AM,1374,20.1,NaN,NaN,76.0,NaN,35.0,...,7.05,49.05,False,1,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),NaN,NaN,NaN
3,0093aed58d3a0c546aa5d410343a15964a274e5d,12/01/2022 12:00:00 AM,12/01/2022 12:00:00 AM,233,1.0,NaN,NaN,3.0,77.0,10.0,...,1.24,11.24,False,1,41.965812,-87.655879,POINT (-87.6558787862 41.96581197),41.986712,-87.663416,POINT (-87.6634164054 41.9867117999)
4,00d6bced9351f2c9af8c1d6828ee81c611a5cabe,12/01/2022 12:00:00 AM,12/01/2022 12:15:00 AM,559,1.9,1.703108e+10,1.703132e+10,8.0,32.0,17.5,...,2.91,20.41,False,1,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.870607,-87.622173,POINT (-87.6221729369 41.8706073724)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6274071,ff7937cf6a52be1e2ae2655dbdfdc790c4fc6372,12/31/2022 11:45:00 PM,12/31/2022 11:45:00 PM,462,2.4,NaN,NaN,66.0,66.0,7.5,...,3.36,10.86,False,1,41.771849,-87.695666,POINT (-87.695666342 41.7718485152),41.771849,-87.695666,POINT (-87.695666342 41.7718485152)
6274072,ff7e7414e84df69cda5d7206680d8d0f11395385,12/31/2022 11:45:00 PM,01/01/2023 12:00:00 AM,485,0.7,1.703108e+10,1.703108e+10,8.0,8.0,7.5,...,1.26,8.76,False,1,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.893216,-87.637844,POINT (-87.6378442095 41.8932163595)
6274073,ffaf8c95783e18b92a15d2f25ab58a31d36f1024,12/31/2022 11:45:00 PM,12/31/2022 11:45:00 PM,293,1.1,1.703124e+10,1.703124e+10,24.0,24.0,5.0,...,2.91,7.91,False,1,41.892658,-87.652534,POINT (-87.6525344838 41.8926581076),41.908379,-87.670945,POINT (-87.6709450752 41.9083786693)
6274074,ffd1373bf649fa526e7e224d96dabe1c6d7e1aa3,12/31/2022 11:45:00 PM,01/01/2023 12:00:00 AM,850,6.6,1.703124e+10,1.703133e+10,24.0,33.0,10.0,...,3.40,13.40,False,1,41.892658,-87.652534,POINT (-87.6525344838 41.8926581076),41.859350,-87.617358,POINT (-87.6173580061 41.859349715)


In [4]:
rs.shape

(6274076, 21)

In [5]:
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6274076 entries, 0 to 6274075
Data columns (total 21 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Trip ID                     object 
 1   Trip Start Timestamp        object 
 2   Trip End Timestamp          object 
 3   Trip Seconds                int64  
 4   Trip Miles                  float64
 5   Pickup Census Tract         float64
 6   Dropoff Census Tract        float64
 7   Pickup Community Area       float64
 8   Dropoff Community Area      float64
 9   Fare                        float64
 10  Tip                         float64
 11  Additional Charges          float64
 12  Trip Total                  float64
 13  Shared Trip Authorized      bool   
 14  Trips Pooled                int64  
 15  Pickup Centroid Latitude    float64
 16  Pickup Centroid Longitude   float64
 17  Pickup Centroid Location    object 
 18  Dropoff Centroid Latitude   float64
 19  Dropoff Centroid Long

In [6]:
rs.describe()

,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tip,Additional Charges,Trip Total,Trips Pooled,Pickup Centroid Latitude,Pickup Centroid Longitude,Dropoff Centroid Latitude,Dropoff Centroid Longitude
count,6.274076e+06,6.274074e+06,3.667219e+06,3.656689e+06,5.741285e+06,5.723860e+06,6.254675e+06,6.254675e+06,6.254675e+06,6.254675e+06,6.274076e+06,5.757798e+06,5.757798e+06,5.739465e+06,5.739465e+06
mean,1.065571e+03,6.820058e+00,1.703138e+10,1.703140e+10,2.738140e+01,2.822174e+01,1.737151e+01,1.197544e+00,4.603747e+00,2.317280e+01,1.027320e+00,4.188886e+01,-8.767109e+01,4.188972e+01,-8.767452e+01
std,7.645197e+02,7.651802e+00,3.412955e+05,3.479832e+05,2.160310e+01,2.215943e+01,1.257279e+01,2.799646e+00,4.253413e+00,1.607519e+01,1.851877e-01,6.878348e-02,6.847579e-02,6.857120e-02,7.419293e-02
min,1.000000e+00,0.000000e+00,1.703101e+10,1.703101e+10,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,4.165022e+01,-8.791362e+01,4.165022e+01,-8.791362e+01
25%,5.290000e+02,1.900000e+00,1.703108e+10,1.703108e+10,8.000000e+00,8.000000e+00,1.000000e+01,0.000000e+00,2.910000e+00,1.313000e+01,1.000000e+00,4.187061e+01,-8.768913e+01,4.187102e+01,-8.769400e+01
50%,8.650000e+02,4.000000e+00,1.703124e+10,1.703128e+10,2.400000e+01,2.500000e+01,1.500000e+01,0.000000e+00,3.550000e+00,1.836000e+01,1.000000e+00,4.189322e+01,-8.765430e+01,4.189322e+01,-8.765588e+01
75%,1.390000e+03,9.000000e+00,1.703183e+10,1.703183e+10,3.400000e+01,3.500000e+01,2.250000e+01,0.000000e+00,5.340000e+00,2.794000e+01,1.000000e+00,4.193571e+01,-8.763172e+01,4.193616e+01,-8.763172e+01
max,4.555300e+04,3.782000e+02,1.703198e+10,1.703198e+10,7.700000e+01,7.700000e+01,8.100000e+02,9.800000e+01,2.811500e+02,8.150000e+02,6.000000e+00,4.202122e+01,-8.752995e+01,4.202122e+01,-8.752995e+01


In [7]:
rs.loc[(rs.duplicated(subset=['Trip ID'], keep='first') == True)] #checking for duplicate ids

,Trip ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,...,Additional Charges,Trip Total,Shared Trip Authorized,Trips Pooled,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location


In [8]:
#rs_clean = rs.loc[(rs_clean.duplicated(subset=['Trip ID'], keep='first') == False)] 
#removes duplicate values of trip id

In [9]:
#filters out trips shorter than 2 miles and longer than 1 day
rs_clean = rs.loc[(rs['Trip Miles'] >= 2) & (rs['Trip Seconds'] <= 86400)] 
rs_clean.shape

(4661630, 21)

In [10]:
rs_clean.to_csv('data/Chicago/cleaned/rideshare/rs_clean_M12.csv', index=False)
#exports clean dataset to csv

In [11]:
rs_clean['Trip Start Timestamp'].min()

'12/01/2022 01:00:00 AM'

In [12]:
rs_clean['Trip End Timestamp'].max()

'12/31/2022 12:45:00 PM'